In [38]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
from konlpy.tag import Okt
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
pd.set_option('display.unicode.east_asian_width', True)

In [39]:
df_test = pd.read_csv('./crawling/test_naver_news_02.csv', index_col = 0)
df_test['category'] = 'Real_Estate'
print(df_test.head())


                                                                         title  \
0                                                                                
1             대단지 대세 열풍에  e편한세상 영종국제도시 센텀베뉴 까지 가세...   
2             포스코건설  거제시에  더샵거제디클리브  아파트 3월 분양   ...      
3             인천 서구의원들  검단 일원 개발행위허가제한에 반대 성명 발표 ...   
4             주택공급대책에 수도권 광역시 분양경기 기대감 유지        ...       

      category  
0  Real_Estate  
1  Real_Estate  
2  Real_Estate  
3  Real_Estate  
4  Real_Estate  


In [40]:
stopwords = pd.read_csv('../datasets/stopwords.csv')

In [41]:
model = load_model('./news_finance.h5')

In [42]:
with open('./news_token.pickle', 'rb') as f:
    token = pickle.load(f)

In [43]:
print(df_test.iloc[7])

title                  강남권 재건축 아파트값 상승률  12 19대책 이후 최고    ...
category                                          Real_Estate
Name: 7, dtype: object


In [44]:
test_title = df_test.iloc[3]['title']
print(test_title)

okt = Okt()
test_title = okt.morphs(test_title)
result = []

for i in range(len(test_title)):
    if test_title[i] not in list(stopwords['stopword']):
        result.append(test_title[i])
test_title = result
test_title = ' '.join(test_title)
print(test_title)

           인천 서구의원들  검단 일원 개발행위허가제한에 반대 성명 발표          
인천 서 의원 검단 일원 개발 행위 허가 제한 반대 성명 발표


In [45]:
# padding

test_title = token.texts_to_sequences([test_title])
test_title = pad_sequences(test_title, 17)
print(test_title)

[[   0    0    0    0    0    0 1116    7 1432 9551  197 1669  610  184
  1230 3138  242]]


In [46]:
label = ['Policy', 'Bond', 'Global_Economy', 'China_Economy', 'IB', 'Stock', 'Real_Estate']
label.sort()
print(label)

['Bond', 'China_Economy', 'Global_Economy', 'IB', 'Policy', 'Real_Estate', 'Stock']


In [47]:
predict = model.predict(test_title)
print(predict)
print(label[np.argmax(predict)]) 

[[0.00268742 0.36334613 0.0244261  0.01522396 0.00336063 0.59002256
  0.00093322]]
Real_Estate


In [48]:
# 하나씩 말고 전체 파일로 test하기
# 데이터프레임 객체에서 인덱싱 -> 리스트화

stopwords = list(pd.read_csv('../datasets/stopwords.csv')['stopword'])
test_titles = []
predicts = []

for title in df_test['title']:
    # 형태소
    title = okt.morphs(title)
    result = []
    
    # 불용어, 한 글자 처리
    for word in title:
        if (word not in stopwords) and len(word) > 1:
            result.append(word)
    title = result
    title = ' '.join(title)
    
    # 토크나이징 (모델 기준 패딩)
    title = token.texts_to_sequences([title])
    title = pad_sequences(title, 17)
    
    # 예측
    predict = model.predict(title)
    predicts.append(label[np.argmax(predict)])

In [49]:
# 예측 결과를 새로운 칼럼으로 추가
df_test['predict'] = predicts
print(df_test.head())

                                                                         title  \
0                                                                                
1             대단지 대세 열풍에  e편한세상 영종국제도시 센텀베뉴 까지 가세...   
2             포스코건설  거제시에  더샵거제디클리브  아파트 3월 분양   ...      
3             인천 서구의원들  검단 일원 개발행위허가제한에 반대 성명 발표 ...   
4             주택공급대책에 수도권 광역시 분양경기 기대감 유지        ...       

      category      predict  
0  Real_Estate       Policy  
1  Real_Estate  Real_Estate  
2  Real_Estate  Real_Estate  
3  Real_Estate  Real_Estate  
4  Real_Estate  Real_Estate  


In [50]:
# 맞으면 O, 틀리면 X 표시하는 칼럼 추가

df_test['OX'] = 0
for i in range(len(df_test.predict)):
    if df_test.category[i] == df_test.predict[i]:
        df_test.OX[i] = 'O'
    else:
        df_test.OX[i] = 'X'
print(df_test)

                                                                          title  \
0                                                                                 
1              대단지 대세 열풍에  e편한세상 영종국제도시 센텀베뉴 까지 가세...   
2              포스코건설  거제시에  더샵거제디클리브  아파트 3월 분양   ...      
3              인천 서구의원들  검단 일원 개발행위허가제한에 반대 성명 발표 ...   
4              주택공급대책에 수도권 광역시 분양경기 기대감 유지        ...       
..                                                                      ...       
85             맨땅에 숨겨진 미래가치  미래의 부촌 도시개발 각광       ...        
86             한화포레나수원장안 1천63가구 분양 3 3 당 1천843만원  ...           
87              가야산 한라비발디 프리미어  잔여 세대 계약 실시       ...         
88             한국주택협회  무의탁 노인 요양시설에 후원금          ....          
89              허위 계약으로 아파트 신고가  실체 알고보니   팩트체크   ...       

       category      predict OX  
0   Real_Estate       Policy  X  
1   Real_Estate  Real_Estate  O  
2   Real_Estate  Real_Estate  O  
3   Real_Estate  Real_Estate  O  
4   Real_Estate  Real_Estate  

C:\Users\698\anaconda3\envs\AI_dev_env\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\698\anaconda3\envs\AI_dev_env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [51]:
# 얼마나 맞히고 틀렸는지
df_test.OX.value_counts()

# O만 세기
df_test.OX.value_counts()['O']

# 정답률
df_test.OX.value_counts()['O']/len(df_test.OX)

0.7888888888888889

In [52]:
df_test.predict.value_counts()

Real_Estate       71
IB                13
Policy             3
Global_Economy     2
Stock              1
Name: predict, dtype: int64

In [53]:
df_test.OX.value_counts(normalize=True)['O'] * 100

78.88888888888889

In [18]:
df_test.to_csv('./naver_result.csv')